In [127]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import os
import re

In [128]:
def overall_orderdata(url_origi):
    url = url_origi[0:url_origi.find('edit')]+'export?format=xlsx'
    df = pd.read_excel(url,sheet_name = 0)
    return df

In [129]:
overall_orderdata("https://docs.google.com/spreadsheets/d/17l5TKof_ZlDisjIW7ZxN6no24UHKSDvL4EmhDjqyeMI/edit?usp=sharing")

,Today,Weeknum,Small set,Big set,1cha pint,2cha pint,3cha pint,4cha pint,5cha pint
0,2024-07-24,30,12,0,0,0,0,0,0


In [130]:
price_dict = {
    "Small set": 425,
    "Big set": 1236,
    "1cha pint": 309,
    "2cha pint": 309,
    "3cha pint": 309,
    "4cha pint": 309,
    "5cha pint": 469,
    "delivery": 150
}

In [131]:
def detailed_orderdata(url_origi):
    url = url_origi[0:url_origi.find('edit')]+'export?format=xlsx'
    df = pd.read_excel(url,sheet_name = 1)
    df = df.dropna(subset=['Name'])
    df["Total Price"] = df["Order"].apply(lambda x: price_dict[x] if x == "Big set" else price_dict[x] + price_dict["delivery"])
    return df

In [132]:
detailed_df = detailed_orderdata("https://docs.google.com/spreadsheets/d/17l5TKof_ZlDisjIW7ZxN6no24UHKSDvL4EmhDjqyeMI/edit?usp=sharing")
detailed_df

,Date,Weeknum,Name,Address,Tel,Order,Total Price
1,2024-07-24,30,ณัชชา วงศ์มาศา,889/187 บ้านกลางกรุงแกรนด์เวียนนา ถนนพระราม3 แ...,0624656162,Small set,575
2,2024-07-24,30,หยิน,55/457 คอนโด ไอดีโอ วุฒากาศ ชั้น 18 ตรอกวิ่งวั...,0942895232,Small set,575
3,2024-07-24,30,Sarah Vinton,"MSI III Garden Apartment. Unit 241, 49/1 Sukhu...",0992306611,Small set,575
4,2024-07-24,30,นันทนัช,บ้านเลขที่49​/บ้านเลขที่เดิม11/224 หมู่บ้านอยู...,0836666142,Small set,575
5,2024-07-24,30,Knight,"บ้านคุณชัยสิทธิ์ ศรีนครินทร์, Srinagarindra Rd...",0818367877,Small set,575
6,2024-07-24,30,Chris,288/105 The Grand Wongwaen Bangna Dokmai Prawe...,0811547836,Small set,575
7,2024-07-24,30,Posh,48/4 วิรัลพัชร เฉลิมพระเกียรติ 48 แยก 1 ถนนเฉล...,084-659-7827,Small set,575
8,2024-07-24,30,นันทรัตน์ ฉันทะกิจ,168/145 หมู่บ้านสารินซิตี้ โครงการ The Canal ห...,0922625127,Small set,575
9,2024-07-24,30,ณัฐสินี พลวิภาต,754/28 ม.เดอะแพลนท์ ซ.พัฒนาการ 38 ถ.พัฒนาการ ส...,0815852455,Small set,575
10,2024-07-24,30,พีรดา สินธุพันธ์ประทุม,129/227 ม.เพอร์เฟคเพลส ซอย 12 หมู่ 3 ถนน รัตนา...,086-338-6698,Small set,575


In [133]:
def clean_address(address):
    if isinstance(address, str):
        # Find the last 5 characters that are digits
        match = re.search(r'(\d{1,5})\D*$', address)
        if match:
            return match.group(1).zfill(5)
    return "00000"

# Apply the function to the Address column
detailed_df['postal_code'] = detailed_df['Address'].apply(clean_address)
detailed_df['Address'] = detailed_df.apply(lambda row: row['Address'].replace(row['postal_code'], '').strip() if row['postal_code'] != "00000" else row['Address'], axis=1)

detailed_df.iloc[0]['Address']

'889/187 บ้านกลางกรุงแกรนด์เวียนนา ถนนพระราม3 แขวงบางโพงพาง เขตยานนาวา จังหวัดกรุงเทพมหานคร'

In [137]:
def format_address(address, max_line_length=34):
    address= str(address)
    words = address.split()
    formatted_address = ""
    current_line = ""

    for word in words:
        if len(current_line) + len(word) + (1 if current_line else 0) <= max_line_length:
            current_line += (" " if current_line else "") + word
        else:
            formatted_address += current_line + "\n"
            current_line = word

    formatted_address += current_line  # Add the last line
    return formatted_address



In [138]:
def create_image(address, postal_code, name, tel,folder):
    # Create a blank white image
    img = Image.open("ผู้รับ.png")
    
    # Initialize ImageDraw
    d = ImageDraw.Draw(img)
    print(img.size)
    line_spacing = 15
    # Add text to the image
    d.text((1750,850), f"{name}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 250), fill=(0, 0, 0))
    d.text((1750,350), f"{tel}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 250), fill=(0, 0, 0))
    # d.text((800,1270), f"{format_address(address)}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 220), fill=(0, 0, 0))
    y = 1290
    for line in format_address(address).split('\n'):
        d.text((800, y), line, font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 200), fill=(0, 0, 0))
        y += 230 + line_spacing
    d.text((2150,2440), f"{str(postal_code)[0]}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 250), fill=(0, 0, 0))
    d.text((2650,2440), f"{str(postal_code)[1]}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 250), fill=(0, 0, 0))
    d.text((3150,2440), f"{str(postal_code)[2]}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 250), fill=(0, 0, 0))
    d.text((3650,2440), f"{str(postal_code)[3]}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 250), fill=(0, 0, 0))
    d.text((4150,2440), f"{str(postal_code)[4]}", font=ImageFont.truetype('EkkamaiNew-Regular.ttf', 250), fill=(0, 0, 0))
    
    # Save the image
    img.save(f"{folder}/{name}_address.png")

In [139]:
folder = str(detailed_df.iloc[0]['Weeknum'])
if not os.path.isdir(folder):
        os.mkdir(folder)
for index, row in detailed_df.iterrows():
    create_image(row['Address'], row['postal_code'], row['Name'],row['Tel'],folder)

(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
(4660, 2904)
